In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
premise_file = "LicensePremise.csv"
premise_df = pd.read_csv(premise_file)
premise_df.head()

,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2


In [4]:
county_file = "CountyLicenseCount.csv"
county_df = pd.read_csv(county_file)
county_df.head()

,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


In [11]:
pd.merge(county_df, premise_df, on = "County ID Code")

,ID,County Name (Licensee),County ID Code,License Count,License Serial Number,Premises Name,License Certificate Number,License Received Date
0,0,ALBANY,5,77,2213243,SRINIDHI LLC,NaN,2018-06-29T00:00:00
1,0,ALBANY,5,77,2211443,QUICKY MART INC,NaN,2018-07-12T00:00:00
2,0,ALBANY,5,77,2211875,ABOALLRGAL CORPORATION,NaN,2018-04-16T00:00:00
3,0,ALBANY,5,77,2211892,VINEYARD ELEVEN CELLARS LLC,898144.0,2018-04-16T00:00:00
4,0,ALBANY,5,77,2211904,THOMAS RAYMOND & BRENDA LEE SIMONEAU,898143.0,2018-04-16T00:00:00
...,...,...,...,...,...,...,...,...
1858,59,WESTCHESTER,20,73,1311918,CHIPOTLE MEXICAN GRILL OF COLORADO LLC,NaN,2018-07-11T00:00:00
1859,60,WYOMING,56,2,3161162,DEBRA A CARTWRIGHT,NaN,2018-06-08T00:00:00
1860,60,WYOMING,56,2,3161162,DEBRA A CARTWRIGHT,NaN,2018-06-08T00:00:00
1861,60,WYOMING,56,2,3161210,ATTICA UNLIMITED ENTERPRISES LLC,NaN,2018-06-19T00:00:00


In [5]:
premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]

In [6]:
premise_transformed = premise_df[premise_cols].copy()
premise_transformed = premise_transformed.rename(columns = {"License Serial Number":"id",
                                                           "Premises Name": "premise_name",
                                                           "County ID Code": "county_id"})
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)
premise_transformed.head()

,premise_name,county_id
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1


In [7]:
premise_transformed.columns

Index(['premise_name', 'county_id'], dtype='object')

In [9]:
county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
county_transformed = county_df[county_cols].copy()

In [10]:
county_transformed = county_transformed.rename(columns={"ID": "id", 
                                                       "County Name (Licensee)":"county_name",
                                                       "License Count":"license_count",
                                                       "County ID Code": "county_id"})
county_transformed.set_index("id", inplace=True)
county_transformed.head()

,county_name,county_id,license_count
id,,,
0,ALBANY,5,77
1,ALLEGANY,59,4
2,BRONX,0,104
3,BROOME,35,14
4,CATTARAUGUS,41,9


In [ ]:
connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgressql://{connection_string}')

In [ ]:
premise_transformed.to_sql(name = "premise", con=engine, if_exists = 'append', index=True)